In [72]:
# !pip3 install pyarrow

In [2]:
from pyspark.sql import SparkSession

In [3]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.types as T
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

In [4]:
spark=SparkSession.builder.appName('read data through spark').getOrCreate()

In [5]:
spark

# Load and clean Paper DF

In [6]:
### load paper into schema
dtypes = pd.read_csv('./schemas/paper.csv').to_records(index=False).tolist()
print(dtypes)
fields = [T.StructField(dtype[0], globals()[f'{dtype[1]}Type']()) for dtype in dtypes]
schema = StructType(fields)
paper_df = spark.read.option('header', 'true').csv('./assets/parsedData/papers.csv', header=True, schema=schema)

[('paper_id', 'Integer'), ('title', 'String'), ('year', 'Integer')]


In [7]:
paper_df.show()

+--------+--------------------+----+
|paper_id|               title|year|
+--------+--------------------+----+
|      65|                null|null|
|     130|                null|null|
|     195|317424;317425;317573|null|
|     260|                null|null|
|     325|                null|null|
|     390|                null|null|
|     455|                null|null|
|     520|       318368;323493|null|
|     585|                null|null|
|     650|                null|null|
|     715|                null|null|
|     780|318420;319233;319...|null|
|     845|                null|null|
|     910|                null|null|
|     975|67604;318882;3718...|null|
|    1040|                null|null|
|    1105|289087;318014;318...|null|
|    1170|                null|null|
|    1235|                null|null|
|    1300|                null|null|
+--------+--------------------+----+
only showing top 20 rows



22/01/17 19:10:42 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 2, schema size: 3
CSV file: file:///Users/yarycka/Desktop/WS2021/DIA/citation-networks-dia/assets/parsedData/papers.csv


In [8]:
### data cleaning for paper schema

### remove spaces from values of the columns
paper_df = paper_df.withColumn("paper_id", trim(paper_df.paper_id))
paper_df = paper_df.withColumn("title", trim(paper_df.title))
paper_df = paper_df.withColumn("year", trim(paper_df.year))

In [9]:
### check for the data types
paper_df.printSchema()
### change the data type of year to Integer
paper_df = paper_df.withColumn("year",paper_df["year"].cast(IntegerType()))

root
 |-- paper_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)



In [10]:
### check for nonsense null data
null_values_paper_df = paper_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in paper_df.columns]
   )
### save the ids of papers whose title is missing to clean up the other dataframes
null_paper_ids = paper_df.filter(paper_df['title'].isNull())
null_paper_ids_list=null_paper_ids.select('paper_id').rdd.flatMap(lambda x: x).collect()
null_paper_ids_list = [int(item) for item in null_paper_ids_list]

22/01/17 19:10:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: paper_id, ref_ids
 Schema: paper_id, title
Expected: title but found: ref_ids
CSV file: file:///Users/yarycka/Desktop/WS2021/DIA/citation-networks-dia/assets/parsedData/papers.csv


In [11]:
### after checking the below dataframes, all papers whose title is missing have the authors besides paper_id = 748056
### decision: fill missing titles with : Missing Title

paper_df=paper_df.na.fill('Missing Title', ['title'])

In [12]:
paper_df.columns

['paper_id', 'title', 'year']

In [13]:
### check if there are duplicate rows
paper_df.join(paper_df.groupBy(paper_df.columns).agg((F.count("*")>1).cast("int").alias("Duplicate_indicator")),
on=paper_df.columns,how="inner").show()
###there are no duplicates
paper_df.groupby(['paper_id']).count().where('count > 1').sort('count', ascending=False).show()

22/01/17 19:10:48 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 2, schema size: 3
CSV file: file:///Users/yarycka/Desktop/WS2021/DIA/citation-networks-dia/assets/parsedData/papers.csv
22/01/17 19:10:50 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 2, schema size: 3
CSV file: file:///Users/yarycka/Desktop/WS2021/DIA/citation-networks-dia/assets/parsedData/papers.csv


+--------+-----+----+-------------------+
|paper_id|title|year|Duplicate_indicator|
+--------+-----+----+-------------------+
+--------+-----+----+-------------------+



+--------+-----+
|paper_id|count|
+--------+-----+
+--------+-----+



# Load and clean Affiliations df

In [14]:
### load affiliation into schema
dtypes = pd.read_csv('./schemas/affiliation.csv').to_records(index=False).tolist()
print(dtypes)
fields = [StructField(dtype[0], globals()[f'{dtype[1]}Type']()) for dtype in dtypes]
schema = StructType(fields)
affiliation_df = spark.read.option('header', 'true').csv('./assets/parsedData/affiliations.csv', header=True, schema=schema)

[('affiliations', 'String'), ('paper_id', 'Integer')]


In [15]:
affiliation_df = affiliation_df.withColumn("affiliations", trim(affiliation_df.affiliations))
affiliation_df = affiliation_df.withColumn("paper_id", trim(affiliation_df.paper_id))
affiliation_df = affiliation_df.withColumn("paper_id",affiliation_df["paper_id"].cast(IntegerType()))

affiliation_df.printSchema()
affiliation_df.show()

root
 |-- affiliations: string (nullable = true)
 |-- paper_id: integer (nullable = true)

+--------------------+--------+
|        affiliations|paper_id|
+--------------------+--------+
|The Queen's Unive...|      65|
|Univ. of Karlsruh...|     130|
|AERE Harwell Labo...|     195|
|University of Mic...|     260|
|Oslo politikammer...|     325|
|Harvard Univ., Ca...|     390|
|Cornell Univ., It...|     455|
|IBM General Techn...|     520|
|               -;-;-|     585|
|New York Univ., N...|     650|
|                   -|     715|
|Xerox Palo Alto R...|     780|
|Univ. of Californ...|     845|
|University of Bol...|     910|
|AT & T Bell Labor...|     975|
|Cornell Univ., It...|    1040|
|University of Mar...|    1105|
|Laboratoire de Ps...|    1170|
|Yale Univ., New H...|    1235|
|                 -;-|    1300|
+--------------------+--------+
only showing top 20 rows



In [16]:
### check for null values in the affiliations column
null_values_affiliations=affiliation_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in affiliation_df.columns]
   )

In [17]:
### This df is used to count papers per unique affiliation, so if the affiliation is missing, it doesnt make sense
### drop all rows where affiliation is null

affiliation_df=affiliation_df.na.drop(how="any", subset=['affiliations'])

In [18]:
affiliation_df.filter(affiliation_df.affiliations.contains('-')).collect()

[Row(affiliations='-;-;-', paper_id=585),
 Row(affiliations='-', paper_id=715),
 Row(affiliations='-;-', paper_id=1300),
 Row(affiliations='-', paper_id=1430),
 Row(affiliations='-', paper_id=1495),
 Row(affiliations='-', paper_id=1625),
 Row(affiliations='-;-;-', paper_id=1690),
 Row(affiliations='Univ. of Illinois at Urbana-Champaign, Urbana;Massachusetts Institute of Technology, Cambridge', paper_id=2795),
 Row(affiliations='-', paper_id=2860),
 Row(affiliations='Mihailo Pupin Institute, Belgrade, Yugoslvia;Carnegie-Mellon Univ., Pittsburgh, PA;Intel Corp., Aloha, OR', paper_id=2990),
 Row(affiliations='Boston Univ., Boston, MA;-', paper_id=3185),
 Row(affiliations='NCR Corporation;-', paper_id=3315),
 Row(affiliations='-', paper_id=3575),
 Row(affiliations='-', paper_id=3705),
 Row(affiliations='-;-', paper_id=4095),
 Row(affiliations='-', paper_id=4160),
 Row(affiliations='Univ. of Alabama, Birmingham;-;-', paper_id=4290),
 Row(affiliations='-', paper_id=4485),
 Row(affiliations='

In [19]:
### check if affiliations are missing as well for the ids whose title was missing in paper_df
#for rows in affiliation_df.select("affiliations","paper_id").collect():
 #   if rows[1] in null_paper_ids_list:
  #      print(rows[0], rows[1])

In [20]:
### split affiliations so we can have clean data and seperate records {paper_id; affiliations}
unique_affiliations_df = affiliation_df.select(F.col("paper_id"), F.explode(F.split(F.col("affiliations"),";")).alias("affiliation"))
unique_affiliations_df.show(20, False)
affiliation_df.show(20, False)

+--------+------------------------------------------------------+
|paper_id|affiliation                                           |
+--------+------------------------------------------------------+
|65      |The Queen's University of Belfast, Belfast, UK        |
|65      |The Queen's University of Belfast, Belfast, UK        |
|130     |Univ. of Karlsruhe, Karlsruhe, West Germany           |
|195     |AERE Harwell Laboratory, Oxon, UK                     |
|195     |Queen's Univ., Belfast, Northern Ireland              |
|260     |University of Michigan, Ann Arbor, MI                 |
|260     |University of Michigan, Ann Arbor, MI                 |
|325     |Oslo politikammer, Oslo, Norway                       |
|390     |Harvard Univ., Cambridge, MA                          |
|390     |Boston Univ., Boston, MA                              |
|455     |Cornell Univ., Ithaca, NY                             |
|455     |Cornell Univ., Ithaca, NY                             |
|520     |

In [21]:
### check for special nonsense characters "-", If the affiliation is missing, there is no point of keeping the rows
###unique_affiliations_df.filter(unique_affiliations_df.affiliations=='-').collect()
unique_affiliations_df=unique_affiliations_df.where(unique_affiliations_df.affiliation!='-')

In [22]:
unique_affiliations_df.show()

+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|      65|The Queen's Unive...|
|      65|The Queen's Unive...|
|     130|Univ. of Karlsruh...|
|     195|AERE Harwell Labo...|
|     195|Queen's Univ., Be...|
|     260|University of Mic...|
|     260|University of Mic...|
|     325|Oslo politikammer...|
|     390|Harvard Univ., Ca...|
|     390|Boston Univ., Bos...|
|     455|Cornell Univ., It...|
|     455|Cornell Univ., It...|
|     520|IBM General Techn...|
|     520|IBM Research Divi...|
|     520|IBM Research Divi...|
|     520|IBM Research Divi...|
|     650|New York Univ., N...|
|     780|Xerox Palo Alto R...|
|     780|Xerox Palo Alto R...|
|     780|Xerox Palo Alto R...|
+--------+--------------------+
only showing top 20 rows



In [23]:
### check for duplicate rows:
unique_affiliations_df.groupby(['paper_id', 'affiliation']).count().where('count > 1').sort('count', ascending=False).show()

+--------+--------------------+-----+
|paper_id|         affiliation|count|
+--------+--------------------+-----+
|  569905|IBM and Universit...|   91|
| 1202294|Open Grid Forum—G...|   88|
| 1542970|University of Ten...|   65|
|  418817|Humanoid Robotics...|   62|
| 1731577|IBM Semiconductor...|   59|
|  772121|IBM Research Divi...|   52|
| 1038111|IBM Thomas J. Wat...|   46|
| 1241693|INFN-CNAF V.le Be...|   44|
| 1077644|Carnegie Mellon U...|   31|
| 1633898|NASA Goddard Spac...|   31|
|  994444|Lehrstuhl fur Ope...|   29|
| 1229219|    No Affiliations,|   29|
| 1210078|IMEC, Kapeldreef ...|   29|
|  864278|Dept. of Electr. ...|   29|
| 1423217|Shanghai Astronom...|   28|
|  827034|The Artist Educat...|   28|
|  771289|IBM Research Divi...|   28|
| 1312394|Atheros Communica...|   27|
|  963134|D. E. Shaw Resear...|   27|
| 1625041|           Microsoft|   27|
+--------+--------------------+-----+
only showing top 20 rows



In [24]:
### drop duplicate rows since here we need unique affiliations
unique_affiliations_df=unique_affiliations_df.dropDuplicates()


In [25]:
unique_affiliations_df.count()

1876283

# Load and clean paper_authors

In [26]:
### load paper_authors into schema
dtypes = pd.read_csv('./schemas/paper_authors.csv').to_records(index=False).tolist()
print(dtypes)
fields = [StructField(dtype[0], globals()[f'{dtype[1]}Type']()) for dtype in dtypes]
schema = StructType(fields)
paper_author_df = spark.read.option('header', 'true').csv('./assets/parsedData/paper_authors.csv', header=True, schema=schema)

[('authors', 'String'), ('paper_id', 'Integer')]


In [27]:
### remove leadind and trailing spaces
paper_author_df = paper_author_df.withColumn("authors", trim(paper_author_df.authors))
paper_author_df = paper_author_df.withColumn("paper_id", trim(paper_author_df.paper_id))

### change data type for paper_id to Integer
paper_author_df = paper_author_df.withColumn("paper_id",paper_author_df["paper_id"].cast(IntegerType()))

paper_author_df.show()
paper_author_df.printSchema()

+--------------------+--------+
|             authors|paper_id|
+--------------------+--------+
| K Devine;F J. Smith|      65|
|J Wolff von Guden...|     130|
|J. K. Reid;A. Jen...|     195|
|William G. Golson...|     260|
|    Stein Schjolberg|     325|
|W Ian Gasarch;Ste...|     390|
|Sam Toueg;Özalp B...|     455|
|Frederick H. Dill...|     520|
|A. R. Calderbank;...|     585|
|         Uzi Vishkin|     650|
|      Stephen S. Yau|     715|
|Michael D. Schroe...|     780|
|         S L. Graham|     845|
|D Maio;M R. Scala...|     910|
|         Pamela Zave|     975|
|G. Salton;E. Voor...|    1040|
|Douglas D. Dunlop...|    1105|
|Patrick Peruch;Vi...|    1170|
| Robert J. Sternberg|    1235|
|Curtis Roads;John...|    1300|
+--------------------+--------+
only showing top 20 rows

root
 |-- authors: string (nullable = true)
 |-- paper_id: integer (nullable = true)



In [28]:
null_values_paper_authors=paper_author_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in paper_author_df.columns]
   )

In [29]:
### check if authors are missing as well for the ids whose title was missing in paper_df
#for rows in paper_author_df.select("authors","paper_id").collect():
 #   if rows[1] in null_paper_ids_list:
  #      print(rows[0], rows[1])

In [30]:
### split authors so we can have clean data and seperate records {paper_id; author}
unique_paper_author_df = paper_author_df.select(F.col("paper_id"), F.explode(F.split(F.col("authors"),";")).alias("author"))
unique_paper_author_df.show(20, False)
paper_author_df.show(20, False)

+--------+---------------------+
|paper_id|author               |
+--------+---------------------+
|65      |K Devine             |
|65      |F J. Smith           |
|130     |J Wolff von Gudenberg|
|195     |J. K. Reid           |
|195     |A. Jennings          |
|260     |William G. Golson    |
|260     |William C. Rounds    |
|325     |Stein Schjolberg     |
|390     |W Ian Gasarch        |
|390     |Steven Homer         |
|455     |Sam Toueg            |
|455     |Özalp Babaoğlu       |
|520     |Frederick H. Dill    |
|520     |Satish Gupta         |
|520     |Daniel T. Ling       |
|520     |Richard E. Matick    |
|585     |A. R. Calderbank     |
|585     |E. G. Coffman, Jr.   |
|585     |L. Flatto            |
|650     |Uzi Vishkin          |
+--------+---------------------+
only showing top 20 rows

+---------------------------------------------------------------+--------+
|authors                                                        |paper_id|
+-------------------------------

In [31]:
### check for duplicate rows:
unique_paper_author_df.groupby(['paper_id', 'author']).count().where('count > 1').sort('count', ascending=False).show()

22/01/17 19:11:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:17 WARN RowBasedKeyValueBatch: Calling spill() on

+--------+-------------------+-----+
|paper_id|             author|count|
+--------+-------------------+-----+
| 1523221|       Dongkun Shin|    4|
| 2059316|      Han Chuanfeng|    3|
| 1202294|          N. Sharma|    3|
| 2042230|            Lu Leng|    3|
| 2040206|       Anchun Cheng|    3|
| 2040206|       Mingshu Wang|    3|
| 1070928|    Federico Rajola|    2|
| 1198797|   Alladi Venkatesh|    2|
| 1936484|      Jiangxia Duan|    2|
| 1071003|Claudia Steinberger|    2|
| 1939475|         Mihai Lupu|    2|
| 1864915| Zakariya Belkhamza|    2|
| 1895212|     Anders Mattson|    2|
| 1071049|         Ming Zhang|    2|
| 1167730|    Max A. Woodbury|    2|
| 1612982|           A. Klemm|    2|
|  581650|  J. Howard Johnson|    2|
|  632381|              J. Li|    2|
| 1058643|         Frank Thun|    2|
| 1070401|      Arieh Iserles|    2|
+--------+-------------------+-----+
only showing top 20 rows



In [32]:
### drop duplicate rows since here we need unique paper-author relation
unique_paper_author_df=unique_paper_author_df.dropDuplicates()

In [33]:
unique_paper_author_df.show(truncate=False)

+--------+-------------------+
|paper_id|author             |
+--------+-------------------+
|2080    |Matthew L. Ginsberg|
|2996    |Martin T. Sullivan |
|3185    |William B. Robinson|
|4362    |C-T Liou           |
|4424    |Ron M Roth         |
|6181    |W J Baggaley       |
|6247    |Ravi B Boppana     |
|6700    |Tomas Hirschfeld   |
|6833    |H D Mills          |
|7480    |S. F. Smith        |
|7868    |R Garcia           |
|8776    |G V Nikolaeva      |
|9492    |B Buchanan         |
|10663   |D Beringer         |
|11576   |A H Zemanian       |
|12221   |V Cody             |
|12936   |L Tondl            |
|13852   |M Gregory Maxwell  |
|13977   |S Reorda           |
|14109   |A Barbour          |
+--------+-------------------+
only showing top 20 rows



# Load and clean Publication_venues df

In [34]:
### load publication_venues into schema
dtypes = pd.read_csv('./schemas/publication_venues.csv').to_records(index=False).tolist()
print(dtypes)
fields = [StructField(dtype[0], globals()[f'{dtype[1]}Type']()) for dtype in dtypes]
schema = StructType(fields)
publication_venue_df = spark.read.option('header', 'true').csv('./assets/parsedData/publication_venues.csv', header=True, schema=schema)

[('paper_id', 'Integer'), ('publication_venue', 'String')]


In [35]:
publication_venue_df = publication_venue_df.withColumn("publication_venue", trim(publication_venue_df.publication_venue))
publication_venue_df = publication_venue_df.withColumn("paper_id", trim(publication_venue_df.paper_id))
publication_venue_df = publication_venue_df.withColumn("paper_id",publication_venue_df["paper_id"].cast(IntegerType()))
publication_venue_df.show()

+--------+--------------------+
|paper_id|   publication_venue|
+--------+--------------------+
|      65|Information Techn...|
|     130|Proc. of the symp...|
|     195|ACM Transactions ...|
|     260|Information and C...|
|     325|Computers and pen...|
|     390|Information and C...|
|     455|SIAM Journal on C...|
|     520|IBM Journal of Re...|
|     585|Journal of the AC...|
|     650|Theoretical Compu...|
|     715|            Computer|
|     780|ACM Transactions ...|
|     845|Methods and tools...|
|     910|Information Proce...|
|     975|ACM Transactions ...|
|    1040|Information Proce...|
|    1105|ACM Transactions ...|
|    1170|Proc. of the 2nd ...|
|    1235|Proc. of the inte...|
|    1300|Foundations of co...|
+--------+--------------------+
only showing top 20 rows



In [36]:
null_values_publication_venue=publication_venue_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in publication_venue_df.columns]
   )

In [37]:
publication_venue_df.filter(publication_venue_df['publication_venue'].isNull()).show()

+--------+-----------------+
|paper_id|publication_venue|
+--------+-----------------+
|  109525|             null|
|  987870|             null|
| 1065415|             null|
| 1555515|             null|
|  144171|             null|
|  804766|             null|
| 1032591|             null|
| 1986272|             null|
| 1013548|             null|
| 1054433|             null|
| 1785618|             null|
|  993855|             null|
| 1444435|             null|
|  893041|             null|
|  947251|             null|
|  638112|             null|
| 1074067|             null|
| 1375797|             null|
| 1376577|             null|
|  861258|             null|
+--------+-----------------+
only showing top 20 rows



# Load and clean Citations df

In [38]:
### load affiliation into schema
dtypes = pd.read_csv('./schemas/citations.csv').to_records(index=False).tolist()
print(dtypes)
fields = [StructField(dtype[0], globals()[f'{dtype[1]}Type']()) for dtype in dtypes]
schema = StructType(fields)
citation_df = spark.read.option('header', 'true').csv('./assets/parsedData/citations.csv', header=True, schema=schema)

[('paper_id', 'Integer'), ('ref_ids', 'String')]


In [39]:
citation_df.show()

+--------+--------------------+
|paper_id|             ref_ids|
+--------+--------------------+
|      65|                null|
|     130|                null|
|     195|317424;317425;317573|
|     260|                null|
|     325|                null|
|     390|                null|
|     455|                null|
|     520|       318368;323493|
|     585|                null|
|     650|                null|
|     715|                null|
|     780|318420;319233;319...|
|     845|                null|
|     910|                null|
|     975|67604;318882;3718...|
|    1040|                null|
|    1105|289087;318014;318...|
|    1170|                null|
|    1235|                null|
|    1300|                null|
+--------+--------------------+
only showing top 20 rows



In [40]:
### remove leading and trailing spaces
citation_df = citation_df.withColumn("ref_ids", trim(citation_df.ref_ids))
citation_df = citation_df.withColumn("paper_id", trim(citation_df.paper_id))
### change data type of paper_id to Integer
citation_df = citation_df.withColumn("paper_id",citation_df["paper_id"].cast(IntegerType()))
citation_df.show()

+--------+--------------------+
|paper_id|             ref_ids|
+--------+--------------------+
|      65|                null|
|     130|                null|
|     195|317424;317425;317573|
|     260|                null|
|     325|                null|
|     390|                null|
|     455|                null|
|     520|       318368;323493|
|     585|                null|
|     650|                null|
|     715|                null|
|     780|318420;319233;319...|
|     845|                null|
|     910|                null|
|     975|67604;318882;3718...|
|    1040|                null|
|    1105|289087;318014;318...|
|    1170|                null|
|    1235|                null|
|    1300|                null|
+--------+--------------------+
only showing top 20 rows



In [41]:
### check for duplicate rows
citation_df.groupby(['paper_id', 'ref_ids']).count().where('count > 1').sort('count', ascending=False).show()

+--------+-------+-----+
|paper_id|ref_ids|count|
+--------+-------+-----+
+--------+-------+-----+



In [42]:
### split citations so we can have clean data and seperate records {paper_id; ref_id}
unique_citation_df = citation_df.select(F.col("paper_id"), F.explode(F.split(F.col("ref_ids"),";")).alias("ref_id"))
unique_citation_df.show(20, False)
citation_df.show(20, False)

+--------+------+
|paper_id|ref_id|
+--------+------+
|195     |317424|
|195     |317425|
|195     |317573|
|520     |318368|
|520     |323493|
|780     |318420|
|780     |319233|
|780     |319290|
|780     |319579|
|780     |320813|
|975     |67604 |
|975     |318882|
|975     |371812|
|975     |404772|
|975     |604232|
|975     |834995|
|975     |836008|
|1105    |289087|
|1105    |318014|
|1105    |318186|
+--------+------+
only showing top 20 rows

+--------+------------------------------------------------+
|paper_id|ref_ids                                         |
+--------+------------------------------------------------+
|65      |null                                            |
|130     |null                                            |
|195     |317424;317425;317573                            |
|260     |null                                            |
|325     |null                                            |
|390     |null                                            |
|4

In [43]:
### change datat type of ref_id to Integer
unique_citation_df = unique_citation_df.withColumn("ref_id",unique_citation_df["ref_id"].cast(IntegerType()))

In [44]:
unique_citation_df.printSchema()

root
 |-- paper_id: integer (nullable = true)
 |-- ref_id: integer (nullable = true)



In [45]:
### check for duplicate rows
unique_citation_df.groupby(['paper_id', 'ref_id']).count().where('count > 1').sort('count', ascending=False).show()

22/01/17 19:11:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/17 19:11:42 WARN RowBasedKeyValueBatch: Calling spill() on

+--------+------+-----+
|paper_id|ref_id|count|
+--------+------+-----+
+--------+------+-----+



In [46]:
## Yaryna's copy-pasted staff
## Should be replaced by Bora's changes
### load author into schema
dtypes = pd.read_csv('./schemas/author.csv').to_records(index=False).tolist()
print(dtypes)
fields = [T.StructField(dtype[0], globals()[f'{dtype[1]}Type']()) for dtype in dtypes]
schema = StructType(fields)
author_df = spark.read.option('header', 'true').csv('./assets/parsedData/authors.csv', header=True, schema=schema)

### remove spaces from values of the columns
author_df = author_df.withColumn("author_id", trim(author_df.author_id))
author_df = author_df.withColumn("citation_count", trim(author_df.citation_count))
author_df = author_df.withColumn("h_index", trim(author_df.h_index))
author_df = author_df.withColumn("name", trim(author_df.name))
author_df = author_df.withColumn("paper_count", trim(author_df.paper_count))

### change data type of author_id, paper_count, citation_count, h_index to Integer
author_df = author_df.withColumn("author_id",author_df["author_id"].cast(IntegerType()))
author_df = author_df.withColumn("citation_count",author_df["citation_count"].cast(IntegerType()))
author_df = author_df.withColumn("h_index",author_df["h_index"].cast(IntegerType()))
author_df = author_df.withColumn("paper_count",author_df["paper_count"].cast(IntegerType()))

### check for nonsense null data
null_values_author_df = author_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in author_df.columns]
   )
null_values_author_df.show()

### Decided to drop rows whose author--name is missing (2 authors)
### At the moment we can evaluate precomputed paper_count and citation_count only if we have the author_names

author_df=author_df.na.drop(how="any", subset=['name'])

### fill empty paper_count, citation_count, h_index to 0   (just one author)
author_df=author_df.na.fill(value=0, subset='paper_count')
author_df=author_df.na.fill(value=0, subset='citation_count')
author_df=author_df.na.fill(value=0, subset='h_index')

## Yaryna
# duplicated_authors_list_y=duplicated_authors_y.select('name').rdd.flatMap(lambda x: x).collect()
duplicated_authors=author_df.groupby(['name'])
unique_authors=duplicated_authors.agg(
    F.avg("paper_count").alias("paper_count"),
    F.avg("citation_count").alias("citation_count"),
    F.avg("h_index").alias("h_index")
)
print(unique_authors.show())

[('author_id', 'Integer'), ('citation_count', 'Integer'), ('h_index', 'Integer'), ('name', 'String'), ('paper_count', 'Integer')]


+---------+--------------+-------+----+-----------+
|author_id|citation_count|h_index|name|paper_count|
+---------+--------------+-------+----+-----------+
|        0|             2|      2|   3|          2|
+---------+--------------+-------+----+-----------+



+--------------------+------------------+------------------+------------------+
|                name|       paper_count|    citation_count|           h_index|
+--------------------+------------------+------------------+------------------+
|        Rui G. Silva|               1.0|               1.0|               1.0|
|           B. Setlik|               1.0|               0.0|               0.0|
|        Hanyue Zhang|               1.0|               0.0|               0.0|
|      Angus Marshall|               1.0|               0.0|               0.0|
|        Dong-hwa Yun|               1.0|               1.0|               1.0|
|        Ryusuke Hata|               1.0|               0.0|               0.0|
|         Peng Zuzeng|               2.0|               0.0|               0.0|
|          T. Hajjari|               3.0|              32.0|               2.0|
|James C. Armstron...|               4.0|               0.5|               0.5|
|             T. Yagi|               2.5

# Run Queries

### Q1.2 Compute paper count per unique affiliation

In [117]:
## Fact table
paper_count_per_affiliation_df = unique_affiliations_df.groupBy('affiliation').count()
print(paper_count_per_affiliation_df.show())

+--------------------+-----+
|         affiliation|count|
+--------------------+-----+
|Massachusetts Ins...|    1|
|             AdaCore|   73|
|Carleton Universi...|  119|
|Academy of Mathem...|    3|
|Advanced Database...|    2|
|National Taiwan U...|  382|
|Computing Laborat...|    8|
|Stony Brook Unive...|  152|
|Department of Ind...|    1|
|Evans and Sutherland|    2|
|Institute for Inf...|   12|
|Horst Görtz Insti...|   32|
|Department of App...|    1|
|      CWI, Amsterdam|   43|
|School of Library...|    1|
|Computer Science ...|    1|
|Department of Ind...|   32|
|University of Man...|    3|
|North China Elect...|    1|
|Department of Com...|    3|
+--------------------+-----+
only showing top 20 rows

None


### Q1.1 Validate precomputed paper counts, citation (ref) counts and h-indexes (per author)

#### Important links
1. Join https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.join.html
2. OrderBy https://sparkbyexamples.com/pyspark/pyspark-orderby-and-sort-explained/#sort


1. Retrieve all publications of the author
2. Calculate the number of references per publication
3. Sort the results in descending order
4. Find a threshold N, where N top publications have at least N references each. N is the h-index of the author.


In [49]:
# Retrieve all publications per author -- retrieved in papers_per_author_df

In [50]:
# Calculate the number of references per publication
refs_per_paper_count_df = unique_citation_df.groupBy("paper_id").count().withColumnRenamed("count","paper_references")
print(refs_per_paper_count_df.show())

+--------+----------------+
|paper_id|paper_references|
+--------+----------------+
|   38220|              10|
|   56680|               4|
|  102960|               1|
|  287495|               8|
|  301535|               4|
|  324155|               5|
|  417040|               6|
|  420485|              17|
|  421135|              67|
|  450125|               4|
|  452010|               5|
|  458185|               4|
|  504400|              11|
|  581815|              11|
|  582400|               3|
|  722735|              23|
|  735215|               3|
|  749060|               7|
|  753025|              18|
|  795275|               2|
+--------+----------------+
only showing top 20 rows

None


In [51]:
# Join and sort the results in descending order
author_papers_with_ref_count = unique_paper_author_df.join(refs_per_paper_count_df, 'paper_id').sort(col("paper_references").desc())
print(author_papers_with_ref_count.show())

+--------+--------------------+----------------+
|paper_id|              author|paper_references|
+--------+--------------------+----------------+
| 2015219|    Michael Kaminsky|             806|
| 2015219|         Mike Dahlin|             806|
|  719353|  Johannes Grabmeier|             772|
|  719353| Volker Weispfenning|             772|
|  719353|      Erich Kaltofen|             772|
| 1221204|                Wang|             555|
| 1583653|          Jörg Meyer|             527|
| 1583653|      Andreas Kerren|             527|
| 1583653|         Achim Ebert|             527|
| 1376720|       Peter Sanders|             524|
| 1376720|          Jop Sibeyn|             524|
| 1376720|        Ulrich Meyer|             524|
| 1221200|         Scott Hauck|             523|
| 1221200|         Andre DeHon|             523|
|  932477| Giovanni De Micheli|             476|
|  932477|         Luca Benini|             476|
| 1504996|      Stefan Schirra|             435|
| 1504996|Matthias M

In [52]:
print(author_papers_with_ref_count.show())

+--------+--------------------+----------------+
|paper_id|              author|paper_references|
+--------+--------------------+----------------+
| 2015219|    Michael Kaminsky|             806|
| 2015219|         Mike Dahlin|             806|
|  719353|      Erich Kaltofen|             772|
|  719353| Volker Weispfenning|             772|
|  719353|  Johannes Grabmeier|             772|
| 1221204|                Wang|             555|
| 1583653|      Andreas Kerren|             527|
| 1583653|         Achim Ebert|             527|
| 1583653|          Jörg Meyer|             527|
| 1376720|        Ulrich Meyer|             524|
| 1376720|       Peter Sanders|             524|
| 1376720|          Jop Sibeyn|             524|
| 1221200|         Scott Hauck|             523|
| 1221200|         Andre DeHon|             523|
|  932477|         Luca Benini|             476|
|  932477| Giovanni De Micheli|             476|
| 1504996|Matthias Muller-H...|             435|
| 1504996|      Stef

In [53]:
window = Window.partitionBy(author_papers_with_ref_count['author']).orderBy(desc("paper_references"), desc("paper_id"))
indexed_grouped_papers_df = author_papers_with_ref_count.select('*', rank().over(window).alias('index'))

In [54]:
h_indexed_papers = indexed_grouped_papers_df.withColumn("possible_h_index", when(indexed_grouped_papers_df.index <= indexed_grouped_papers_df.paper_references, indexed_grouped_papers_df.index).otherwise(0))
print(h_indexed_papers.filter(h_indexed_papers.author == "Antonio García").show(100, False))

+--------+--------------+----------------+-----+----------------+
|paper_id|author        |paper_references|index|possible_h_index|
+--------+--------------+----------------+-----+----------------+
|1021079 |Antonio García|21              |1    |1               |
|1043043 |Antonio García|19              |2    |2               |
|1714191 |Antonio García|13              |3    |3               |
|1442848 |Antonio García|13              |4    |4               |
|1704289 |Antonio García|9               |5    |5               |
|569823  |Antonio García|9               |6    |6               |
|520842  |Antonio García|5               |7    |0               |
|424188  |Antonio García|5               |8    |0               |
|1719840 |Antonio García|4               |9    |0               |
|290424  |Antonio García|4               |10   |0               |
|1591441 |Antonio García|3               |11   |0               |
|1228557 |Antonio García|3               |12   |0               |
|548207  |

In [ ]:
h_indexed_aggregated_papers = h_indexed_papers.groupBy('author')\
    .agg(
        F.count('paper_id').alias("computed_paper_count"),\
        F.sum('paper_references').alias("computed_citation_count"),\
        F.max('possible_h_index').alias("computed_h_index")\ 
    ).withColumnRenamed("author", "name")

In [ ]:
h_indexed_aggregated_papers.filter(h_indexed_aggregated_papers.author.like("%Antonio García%")).show(truncate=False)

In [55]:
## Join the real authors with the validated data from paper dataset
unique_authors_with_validated_cols_df = unique_authors.join(h_indexed_aggregated_papers, 'name')

NameError: name 'h_indexed_aggregated_papers' is not defined

In [ ]:
print(unique_authors_with_validated_cols_df
      .filter(\
    unique_authors_with_validated_cols_df.name.like("%Antonio García%")).show(truncate=False))